In [ ]:
# Run this cell to mount your Google Drive.

from google.colab import drive
drive.mount("/content/drive")

# Imports

In [ ]:
# path
pth = '/content/drive/MyDrive/Colab Notebooks/Thesis'

%cd /content/drive/My Drive/Colab Notebooks/Thesis/SupervisedLearning

from train import *
from visualize_data import *
from utils import *

%cd /content/drive/My Drive/Colab Notebooks/Thesis

# State-Action Pair Guideline

Specify the desired state-action pair:

i.e. Before Pickup, Before Discard: Draw (bpbd_draw)

## Model Name Guideline

Specify the following parameters:

- **Data Selection**:
 - **`state-action pair`**
   - possible state-action pairs
```r
state_action_pair = {'all': 'all', # all actions
                    'bpbd': 'draw', # actions 2/3 
                    'apbd': ['discard', 'knock'], # actions 6-57, 58-109
                    'apad': 'knock_bin'}
```
 - **`model_name`**
   - name of the model based on **pruned states** and **chosen action**
 - **`numGames`**
   - Number of games used to train model
$$numGames \in [2000, 6000, 8000]$$

 - **`pruneStatesList`**
   - which states to omit when training model 
 ```r
 pruneStatesList is a list, {'currHand','topCard','deadCard','oppCard','unknownCard'}
 ```
 - **`actionChoice`**
   - which specific action to train model 
```r
actionChoice is one of {'all','draw','discard','knock'}
```
 - **`balance`** (T/F, **`default = False`**)
   - balance data by smallest class

--- 

- **Model Parameters**:

| Parameter         | Description                | Type             | Default |
| ----------------- |---------------------------:|:----------------:| -------:|
| **batch_size**    | num samples per iteration  | int              | 1000    |
| **learning_rate** | step size of model         | float            | 0.001   |
| **epoch**         | num of iterations          | int              | 100     |
| **model_fnc**     | model selection            | str              | MLP_base|
| **activation**    | model activation fcn       | str              | sig     |
| **loss**          | loss function selection    | str              | MSE     |
| **loss_weight**   | weighted loss (CELoss only)| str              | None    |
| **pre_train**     | initialize model weights   | bool             | False   |
| **model_PT**      | path to pretrained network | str (model/path) | null    |
| **device**        | device to train/test model | str (cpu/cuda)   | cpu     |
| **multi_data_pth**| extra data to train model  | dict             | { }     |

 - **`model_fnc`**
```r
# default
MLP_base = {1 Hidden Layer:
            input_size -> input_size*2 -> output_size,
            Activation (b/w Layers): 'Sigmoid' (default)
            Final Activation: 'Softmax'}
# Additional Models
MLP_2HL = {2 Hidden Layers:
            input_size -> input_size*2 -> input_size*2 -> output_size,
            Activation (b/w Layers): 'Sigmoid' (default)
            Final Activation: 'Softmax'} 
```

 - **`activation`**
```r
activation = {'sig': torch.nn.Sigmoid(), # default
             'relu': torch.nn.ReLU(),
             'tanh': torch.nn.Tanh()}
```

 - **`loss`**
```r
loss = {'MSE': torch.nn.MSELoss(), # default
        'CELoss': torch.nn.CrossEntropyLoss()} 
```

 - **`loss_weight`** (CELoss only)
```r
loss_weight = {None: np.ones(output_size), # default
            'icf': inverse class frequency,
            'log_icf': log inverse class frequency} 
```

 - **`multi_data_pth`**
```r
multi_data_pth =  {'First Path': {'data_pth': {'pth'}/data/{'state'}/{'action'},
                                  'numGames': {number of games}},
                   'Second Path': {'data_pth': {'pth'}/data/{'state'}/{'action'},
                                   'numGames': {number of games}}
                   ...}
# where state/action are legal state_action pairs
# Example:
multi_data_pth = {'First Path': {'data_pth': '{}/data/{}/{}'.format(pth,'apbd','knock'),
                                 'numGames': 8000}}
```

# State-Action Pair

## Model Name

### Parameters

In [ ]:
# state_action pair
state = 'all'
action = 'all'

# model name
model_name = 'all_states_all_actions_CE_WLoss'

# Number of Games
numGames = 8000

# prunable states
# {'currHand','topCard','deadCard','oppCard','unknownCard'} or blank if None
pruneStatesList = []

# choosable actions
# {'all','draw_pickup','discard','knock','knock_bin'}
actionChoice = 'all'

# Balance classes
balance = False

# Training parameters
batch_size = 1000
lr = 0.001
epochs = 100

# choose model architecture, activation function
# {'MLP_base', 'MLP_2HL}, {'sig', 'relu', 'tanh'}
model_fnc = 'MLP_base'
activation = 'sig'

# Loss Function and Class Weights, either inverse class freq, or log icf
# {'MSE', 'CELoss'}, {None, 'icf', 'log_icf'}
loss = 'CELoss'
loss_weight = 'log_icf'

# Pretrain model
pre_train = False
model_PT = ''

# device for model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# additional data paths
# multi_data_pth =  {'First Path': {'data_pth': {'pth'}/data/{'state'}/{'action'},
#                                   'numGames': {number of games}},
#                    'Second Path': {'data_pth': {'pth'}/data/{'state'}/{'action'},
#                                    'numGames': {number of games}}
#                    ...}
# where state/action are legal state_action pairs
# ex. multi_data_pth = {'First Path': {'data_pth': '{}/data/{}/{}'.format(pth,'apbd','knock'),
#                                  'numGames': 8000}}
multi_data_pth = {}

### Create Directories

In [ ]:
data_pth, model_pth, plot_pth = create_dir(pth, state, action, model_name)

### Load, Prune, and Split Training Data

In [ ]:
train_loader, val_loader, weights, classes = load_train_data(data_pth, plot_pth,
                                                             numGames, batch_size, 
                                                             pruneStatesList, actionChoice, 
                                                             balance, loss_weight, multi_data_pth,
                                                             visualize=False)

### Train Model & Visualize Results

#### Train Model

In [ ]:
# Write Parameters
write_params(pth, state, action, 
             model_name, 
             numGames, 
             pruneStatesList, actionChoice, 
             balance, 
             batch_size, lr, epochs, 
             model_fnc, activation, 
             loss, loss_weight, 
             pre_train, model_PT, 
             device,
             multi_data_pth)

In [ ]:
model, model_acc, model_loss = train(train_loader, val_loader, plot_pth, batch_size, lr, epochs, verbose=True,
                                     model_fnc=model_fnc, activation=activation,
                                     loss=loss, weights=weights,
                                     pre_train=pre_train, model_PT=model_PT, device=device)
torch.save(model, '{}/model.pt'.format(model_pth))
torch.save(model_acc, '{}/model_acc.pt'.format(model_pth))
torch.save(model_loss, '{}/model_loss.pt'.format(model_pth))

#### Confusion Matrix

##### Load Models

In [ ]:
model = torch.load('{}/model.pt'.format(model_pth), map_location=device)
model_acc = torch.load('{}/model_acc.pt'.format(model_pth), map_location=device)
model_loss = torch.load('{}/model_loss.pt'.format(model_pth), map_location=device)

##### Train Set

In [ ]:
currGames = 8000
plot_cm(plot_pth, classes, model, train_loader, device, numGames=currGames)

##### Validation Set

In [ ]:
currGames = 8000
plot_cm(plot_pth, classes, model, val_loader, device, numGames=currGames, mode='val')

##### Test Set (6k)

In [ ]:
currGames = 6000
test_loader_6k, classes = load_test_data(data_pth, currGames, 
                                         pruneStatesList, actionChoice)
plot_cm(plot_pth, classes, model, test_loader_6k, device, numGames=currGames)

##### Test Set (2k)

Test on all three models generated:

In [ ]:
currGames = 2000
test_loader_2k, classes = load_test_data(data_pth, currGames, 
                                         pruneStatesList, actionChoice)

###### all epoch

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames)

###### max validation accuracy

In [ ]:
plot_cm(plot_pth, classes, model_acc, test_loader_2k, device, numGames=currGames, mode='acc')

###### min validation loss

In [ ]:
plot_cm(plot_pth, classes, model_loss, test_loader_2k, device, numGames=currGames, mode='loss')

##### Test Set - Class Groups

Test on all three class groups:

```r
class_group = {'draw', 'discard', 'knock'}
```

In [ ]:
currGames = 2000
test_loader_2k, classes = load_test_data(data_pth, currGames, 
                                         pruneStatesList, actionChoice)

###### draw

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames, class_group='draw')

###### discard

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames, class_group='discard')

###### knock

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames, class_group='knock')

##### Test Set - Discard Only (8k)

In [ ]:
currGames = 8000
data_pth_discard = '/content/drive/MyDrive/Colab Notebooks/Thesis/data/apbd/discard'
test_loader_2k, classes = load_test_data(data_pth_discard, currGames, 
                                         pruneStatesList, actionChoice)

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames, mode='knock_data_only')

##### Test Set - Knock Only (8k)

In [ ]:
currGames = 8000
data_pth_knock = '/content/drive/MyDrive/Colab Notebooks/Thesis/data/apbd/knock'
test_loader_2k, classes = load_test_data(data_pth_knock, currGames,
                                         pruneStatesList, actionChoice)

In [ ]:
plot_cm(plot_pth, classes, model, test_loader_2k, device, numGames=currGames, mode='knock_data_only')